In [60]:
from langchain_community.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

import os
import sys

sys.path.append('..')  # add parent dir to sys.path
import utils

In [62]:
# load csv-docs
loader = CSVLoader(
    file_path='datasets/lenta-ru-news-short.csv',
    csv_args={'fieldnames': ['Источник', 'Заголовок', 'Содержание', 'Тема', 'Теги', 'Дата']},
    # content_columns=['Заголовок', 'Содержание', 'Тема', 'Теги', 'Дата'],
    source_column='Источник'
)

docs = loader.load()
docs[:3]

[Document(metadata={'source': 'https://lenta.ru/news/2006/05/24/fire/', 'row': 0}, page_content='Источник: https://lenta.ru/news/2006/05/24/fire/\nЗаголовок: В Стамбуле горит багажное отделение аэропорта "Ататюрк"\nСодержание: \nТема: Мир\nТеги: Все\nДата: 2006/05/24'),
 Document(metadata={'source': 'https://lenta.ru/news/2000/11/06/spielberg/', 'row': 1}, page_content='Источник: https://lenta.ru/news/2000/11/06/spielberg/\nЗаголовок: Британская Академия Искусства Кино и\xa0Телевидения наградила режиссера Стивена Спилберга\nСодержание: Английский принц Эндрю вручил Стивену Спилбергу ("Парк Юрского периода", "Список Шиндлера", "Спасти рядового Райана") награду Британской Академии Искусства Кино и Телевидения (BAFTA). В присутстствии многочисленных звезд Голливуда, а также своей жены Кэйт Капшоу и своего делового партнера Джеффри Катценберга, Спилберг сказал, что влияние британского киноискусства на Голливуд всегда было очень сильным. В этом году награда BAFTA переименована в Stanley Kub

In [63]:
# embeddings model
model_kwargs = {'device': 'mps'} # use GPU
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name='cointegrated/LaBSE-en-ru',
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [64]:
%%time
# create or load vector store

if not os.path.exists('faiss-index'):
    db = FAISS.from_documents(
        documents=docs,
        embedding=embeddings,
    )
    db.save_local(
        folder_path='faiss-index',
        index_name='index'
    )

else:
    db = FAISS.load_local(
        folder_path='faiss-index',
        index_name='index',
        embeddings=embeddings,
        allow_dangerous_deserialization=True,
    )

CPU times: user 28.4 s, sys: 18.8 s, total: 47.2 s
Wall time: 9min 28s


In [65]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': 3, 'score_threshold': 0.2},
)

In [66]:
query = 'Когда будет праздноваться день рождения Лермонтова?'

rel_docs = retriever.invoke(query)

for idx, doc in enumerate(rel_docs, 1):
    print(f'| ЗАПРОС {idx}\n{doc.page_content}\n----------\n')

| ЗАПРОС 1
Источник: https://lenta.ru/news/2001/04/25/birthday/
Заголовок: На юбилее Жириновского трезвых и женатых не будет
Содержание: Лидеру ЛДПР, вице-спикеру Госдумы Владимиру Жириновскому исполнилось 55 лет. С этой датой его поздравили коллеги по депутатскому корпусу. Председатель Госдумы Геннадий Селезнев вручил юбиляру почетную грамоту. Как передает РИА "Новости", с поздравительной речью на заседании палаты выступил член фракции ЛДПР Алексей Митрофанов. "Как всегда, мы будем отмечать день рождения Жириновского в виде тотального народного праздника, приглашение не требуется, трезвых и женатых не будет", - заявил депутат. Сам Жириновский сообщил, что к этому времени уже получил телеграммы от председателя Счетной палаты Сергея Степашина, министра обороны Сергея Иванова, главы Пенсионного фонда Михаила Зурабова, а также многих работников аппарата Госдумы. Праздничный банкет в честь Жириновского состоится вечером в центральном аппарате ЛДПР в Первом Басманном переулке. Там партийный

In [67]:
query = 'Прямой эфир с Владимиром Путиным'

rel_docs = retriever.invoke(query)

for idx, doc in enumerate(rel_docs, 1):
    print(f'| ЗАПРОС {idx}\n{doc.page_content}\n----------\n')

| ЗАПРОС 1
Источник: https://lenta.ru/news/2012/10/31/line/
Заголовок: Путин отказался от "прямой линии"
Содержание: Президент России Владимир Путин в 2012 году не будет общаться с населением в формате "прямой линии", которая ежегодно в течение последних десяти лет проводилась в эфире федеральных каналов. Об этом 31 октября пишет "Коммерсантъ" со ссылкой на пресс-секретаря президента Дмитрия Пескова. Пресс-секретарь заявил, что общение Владимира Путина с населением в текущем году состоится, однако будет иметь другой формат, и добавил, что информация об этом будет доступна в скором времени. По данным "Коммерсанта", в конце ноября Путин планирует провести крупную пресс-конференцию для журналистов. При этом, по словам Пескова, дата послания президента Федеральному собранию пока не известна, поскольку  послание "не может быть проходным, а должно содержать очень серьезные стратегические ориентиры". Глава Международного института политической экспертизы Евгений Минченко, комментируя сообщени

In [68]:
query = 'Во сколько лет начинается пенсионный возраст в России?'

rel_docs = retriever.invoke(query)

for idx, doc in enumerate(rel_docs, 1):
    print(f'| ЗАПРОС {idx}\n{doc.page_content}\n----------\n')

| ЗАПРОС 1
Источник: https://lenta.ru/news/2016/10/26/sobyanin/
Заголовок: Собянин рассказал о рекордной продолжительности жизни москвичей
Содержание: Средняя продолжительность жизни в Москве в настоящее время достигла 77 лет. Этот показатель является рекордным за всю историю наблюдений, рассказал мэр города Сергей Собянин. Его слова приводит РИА Новости. По данным градоначальника за прошедшие шесть лет смертность от инфаркта снизилась в три раза, младенческая и материнская смертность сократилась на 30 процентов. «Еще совсем недавно между подозрением на болезнь и началом лечения проходило более ста дней, что в случае с онкологией совершенно недопустимо. Наша цель – сократить это время до 10 дней», — пояснил Собянин. Он также рассказал о планах правительства города провести модернизацию здравоохранения Москвы. Особое внимание при этом будет уделяться диагностике и лечению инсульта, а также онкологических заболеваний. Вместе с тем, Собянин анонсировал реконструкцию родильных домов в Моск

In [ ]:
query = 'Что такое ВКонтакте?'

rel_docs = retriever.invoke(query)

for idx, doc in enumerate(rel_docs, 1):
    print(f'| ЗАПРОС {idx}\n{doc.page_content}\n----------\n')

| ЗАПРОС 1
Источник: https://lenta.ru/news/2015/05/13/vk/
Заголовок: Мобильное приложение «ВКонтакте» стало самым популярным в России
Содержание: Мобильные приложения «ВКонтакте» для операционных систем iOS и Android возглавили рейтинги TNS в крупных городах России за март 2015 года. Об этом «Ленте.ру» рассказал представитель «ВКонтакте» Альберт Усманов. По данным аналитической компании TNS, среднесуточная аудитория мобильных приложений «ВКонтакте» в российских городах с населением от 700 тысяч жителей превысила 5,6 миллиона человек. Это в 4 раза больше, чем у ближайших конкурентов социальной сети — Facebook и «Одноклассники», и в 18 раз больше, чем у сервиса микроблогов Twitter. «Жители крупных городов России используют мобильные приложения "ВКонтакте" чаще любых других — 18 дней в месяц», — сообщил Усманов. Он добавил, что в среднем в день каждый человек открывает приложение социальной сети 20 раз и проводит в нем более 30 минут. Второе место по дневному охвату аудитории мобильных пр

: 